In [ ]:
import pandas as pd 
import  numpy as np 
import nltk 
import spacy
import string
from bs4 import BeautifulSoup as bs
from collections import Counter
import cv2
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import random
import re
from nltk.stem.porter import PorterStemmer

from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
path = 'C:/Users/Srujan/Documents/Datasets/disaster tweets/'

In [ ]:
tweets = pd.read_csv(path+'train.csv')
tweets.head(100)

In [ ]:
tweets.info()

In [ ]:
tweets.drop(['id','keyword','location'],axis=1,inplace=True)
tweets.head()

In [ ]:
tweets['target'].hist()

In [ ]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because",
                    "could've": "could have", "couldn't": "could not", "didn't": "did not",  
                    "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", 
                    "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", 
                    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  
                    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have",
                    "I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", 
                    "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                    "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
                    "mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", 
                    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", 
                    "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", 
                    "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is",
                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                    "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", 
                    "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", 
                    "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", 
                    "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                    "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

numbers = ['0','1','2','3','4','5','6','7','8','9']

In [ ]:
 class Text_preprocessing:
    
    def __init__(self,data):
        
        self.ps = PorterStemmer()
        self.data = data
        
    def remove_html_tags(self,column):
        
        def remove_html(text):
            soup = bs(text,'lxml')
            html_free = soup.get_text()
            return html_free
        
        return self.data[column].apply(remove_html)
        
    def remove_contractions(self,column):
        
        replace = lambda x: contraction_dict[x] if (x in contraction_dict.keys()) else x
        
        def replace_contractions(text):
            new_text = map(replace,text.split(' '))
            return ' '.join(list(new_text))
    
        return self.data[column].apply(replace_contractions)
        
    def lower(self,column):
        
        return self.data[column].apply(str.lower)
    
    def remove_punctuation(self,column):
        
        def remove_punctuation(text):
            no_punct = "".join([c for c in text if c not in string.punctuation+('।')])    
            return no_punct

        return self.data[column].apply(remove_punctuation)
    
    def remove_numbers(self,column):
        
        def remove_numbers(text):
            no_num = "".join([c for c in text if c not in numbers])    
            return no_num

        return self.data[column].apply(remove_numbers)
    
    def remove_stopwords(self,column):
        
        def remove_stopwords(text):
            words = text.split()
            words = [word for word in words if word not in stopwords.words('english')]
            words = ' '.join(words)
            return words
        
        return self.data[column].apply(remove_stopwords)

In [ ]:
tp = Text_preprocessing(tweets)

tweets['text'] = tp.remove_html_tags('text')
tweets['text'] = tp.remove_contractions('text')
tweets['text'] = tp.lower('text')
tweets['text'] = tp.remove_punctuation('text')
tweets['text'] = tp.remove_numbers('text')
tweets['text'] = tp.remove_stopwords('text')

tweets.head(10)

In [ ]:
tweets.to_csv(path+'processed_train.csv')

In [ ]:
tweets = pd.read_csv(path+'processed_train.csv')

In [ ]:
# max length
max_len = 0
for row in tweets['text']:
    if max_len < len(row):
        max_len = len(row)
print(max_len)

In [ ]:
sentences = list(tweets['text'])

In [ ]:
vocab_size = 500

In [ ]:
onehot_repr=[one_hot(words,vocab_size)for words in sentences] 

In [ ]:
onehot_repr[:5]

# Word embedding representation

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
sent_length = 100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs) 

In [ ]:
model=Sequential()
model.add(Embedding(vocab_size,10,input_length=sent_length))
model.compile('adam','mse')

In [ ]:
print(model.predict(embedded_docs))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
pd.DataFrame(cosine_similarity(model.predict(embedded_docs)[0]))

In [ ]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(50))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)